<a href="https://colab.research.google.com/github/Fredestroyer007/Classify/blob/master/Notebooks/Building_An_Equal_Weight_S%26P_500_Index_Fund.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#Building An Equal-Weight S&P 500 Index Fund

##Library Import

First we import some libraries we will use in this notebook.

In [1]:
!pip install xlsxwriter
!pip install datapackage
!pip install wikipedia

     |████████████████████████████████| 153kB 5.6MB/s 
     |████████████████████████████████| 92kB 4.1MB/s 
     |████████████████████████████████| 81kB 5.2MB/s 
     |████████████████████████████████| 71kB 4.6MB/s 
     |████████████████████████████████| 133kB 6.6MB/s 
     |████████████████████████████████| 245kB 7.4MB/s 
     |████████████████████████████████| 133kB 8.6MB/s 
     |████████████████████████████████| 51kB 5.1MB/s 
     |████████████████████████████████| 7.2MB 7.7MB/s 
     |████████████████████████████████| 71kB 8.1MB/s 
  Created wheel for unicodecsv: filename=unicodecsv-0.14.1-cp36-none-any.whl size=10768 sha256=13ade47d616bdba9405bd0bd596b971920ffa903cae0b9b0891ac328fe2b9225
  Stored in directory: /root/.cache/pip/wheels/a6/09/e9/e800279c98a0a8c94543f3de6c8a562f60e51363ed26e71283
  Created wheel for linear-tsv: filename=linear_tsv-1.1.0-cp36-none-any.whl size=7382 sha256=76dec7b3ced255374c298399c742b402826b84288afb1b73bba56601d73bdb8c
  Stored in directory: /root/.

In [2]:
from datetime import datetime
import numpy as np
import pandas as pd
import requests
import xlsxwriter as xlsx
import math
import os
import re
import shutil
import wikipedia as wp

##Importing Our List of Stocks

###Scrap the S&P Constituents from Wikipedia

Function to scrape the data from wikipedia based on this [github repo](https://github.com/fja05680/sp500).

In [3]:
def get_table(title, filename, match, use_cache=False):

    if use_cache and os.path.isfile(filename):
        pass
    else:
        html = wp.page(title).html()
        df = pd.read_html(html, header=0, match=match)[0]
        
        df.to_csv(filename, header=True, index=False, encoding='utf-8')
            
    df = pd.read_csv(filename)
    return df

Create the dataframe of sp500 securities.

In [4]:
title = 'List of S&P 500 companies'
filename = 'sp500.csv'
sp500 = get_table(title, filename, match='Symbol')

# dd/mm/YY H:M:S
now = datetime.now()
dt_string = now.strftime("%m/%d/%Y %H:%M:%S")
print('{} (retrieved {})'.format(title, dt_string))
sp500

List of S&P 500 companies (retrieved 02/01/2021 20:49:41)


,Symbol,Security,SEC filings,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,CIK,Founded
0,MMM,3M Company,reports,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,66740,1902
1,ABT,Abbott Laboratories,reports,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1800,1888
2,ABBV,AbbVie Inc.,reports,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,1551152,2013 (1888)
3,ABMD,ABIOMED Inc,reports,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,815094,1981
4,ACN,Accenture plc,reports,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1467373,1989
...,...,...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,reports,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1041061,1997
501,ZBRA,Zebra Technologies,reports,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,877212,1969
502,ZBH,Zimmer Biomet,reports,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1136869,1927
503,ZION,Zions Bancorp,reports,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,109380,1873


###Cleanup the data

Cleanup the dataframe

In [9]:
sp500 = sp500.drop('CIK', 1)
sp500 = sp500.drop('SEC filings', 1)
sp500

,Symbol,Security,GICS Sector,GICS Sub-Industry,Headquarters Location,Date first added,Founded
0,MMM,3M Company,Industrials,Industrial Conglomerates,"St. Paul, Minnesota",1976-08-09,1902
1,ABT,Abbott Laboratories,Health Care,Health Care Equipment,"North Chicago, Illinois",1964-03-31,1888
2,ABBV,AbbVie Inc.,Health Care,Pharmaceuticals,"North Chicago, Illinois",2012-12-31,2013 (1888)
3,ABMD,ABIOMED Inc,Health Care,Health Care Equipment,"Danvers, Massachusetts",2018-05-31,1981
4,ACN,Accenture plc,Information Technology,IT Consulting & Other Services,"Dublin, Ireland",2011-07-06,1989
...,...,...,...,...,...,...,...
500,YUM,Yum! Brands Inc,Consumer Discretionary,Restaurants,"Louisville, Kentucky",1997-10-06,1997
501,ZBRA,Zebra Technologies,Information Technology,Electronic Equipment & Instruments,"Lincolnshire, Illinois",2019-12-23,1969
502,ZBH,Zimmer Biomet,Health Care,Health Care Equipment,"Warsaw, Indiana",2001-08-07,1927
503,ZION,Zions Bancorp,Financials,Regional Banks,"Salt Lake City, Utah",2001-06-22,1873


##Acquiring API Token

Get API key for IEX Cloud Sandbox mode.

In [10]:
IEX_CLOUD_API_TOKEN = 'Tpk_059b97af715d417d9f49f50b51b1c448'

##Parsing Our API Call

In [16]:
symbol = 'AAPL'
api_url = f'https://sandbox.iexapis.com/stable/stock/{symbol}/quote/?token={IEX_CLOUD_API_TOKEN}'
data = requests.get(api_url).json()
print(data)

{'symbol': 'AAPL', 'companyName': 'Apple Inc', 'primaryExchange': 'DLRA)ENKNM/EQSLTSAGTEL A( OBASCG ', 'calculationPrice': 'iexlasttrade', 'open': None, 'openTime': None, 'openSource': 'ifalifco', 'close': None, 'closeTime': None, 'closeSource': 'ioifaclf', 'high': None, 'highTime': None, 'highSource': None, 'low': None, 'lowTime': None, 'lowSource': None, 'latestPrice': 137.56, 'latestSource': 'IEX Last Trade', 'latestTime': 'February 1, 2021', 'latestUpdate': 1645900385059, 'latestVolume': None, 'iexRealtimePrice': 137.98, 'iexRealtimeSize': 102, 'iexLastUpdated': 1618169939827, 'delayedPrice': None, 'delayedPriceTime': None, 'oddLotDelayedPrice': None, 'oddLotDelayedPriceTime': None, 'extendedPrice': None, 'extendedChange': None, 'extendedChangePercent': None, 'extendedPriceTime': None, 'previousClose': 137.02, 'previousVolume': 178676925, 'change': 2.2, 'changePercent': 0.01666, 'volume': None, 'iexMarketPercent': 0.013270093716460199, 'iexVolume': 1398390, 'avgTotalVolume': 123805

In [19]:
price = data['latestPrice']
market_cap = data['marketCap']

##Adding Our Stock Data to a Pandas Dataframe

In [22]:
my_columns = ['Ticker', 'Price','Market Capitalization', 'Number Of Shares to Buy']
final_dataframe = pd.DataFrame(columns = my_columns)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy


In [23]:
final_dataframe = final_dataframe.append(
                                        pd.Series(['AAPL', 
                                                   data['latestPrice'], 
                                                   data['marketCap'], 
                                                   'N/A'], 
                                                  index = my_columns), 
                                        ignore_index = True)
final_dataframe

,Ticker,Price,Market Capitalization,Number Of Shares to Buy
0,AAPL,137.56,2259303492842,N/A


##Looping Through The Tickers in Our List of Stocks

In [38]:
columns = list(sp500) 
for i in columns:
  print(sp500[0][i])

KeyError: ignored